In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
import io
import matplotlib.pyplot as plt
import obspy
from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
import re
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import seaborn as sns
import json
from collections import defaultdict

In [ ]:
figure_path = Path("figures")
if not figure_path.exists():
    figure_path.mkdir()
result_path = Path("results")
if not result_path.exists():
    result_path.mkdir()

## DAS Location

In [ ]:
das_location = pd.read_csv("DAS-LAX_coor_tap_test.csv")
maxradius = 2.0
minlongitude = das_location["longitude"].min() - maxradius
maxlongitude = das_location["longitude"].max() + maxradius
minlatitude = das_location["latitude"].min() - maxradius
maxlatitude = das_location["latitude"].max() + maxradius


In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution="10m", color="gray", linewidth=0.5)
terrain = cimgt.Stamen("terrain-background")
ax.add_image(terrain, 10)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False
ax.set_extent(
    [
        das_location["longitude"].min() - 0.5,
        das_location["longitude"].max() + 0.5,
        das_location["latitude"].min() - 0.5,
        das_location["latitude"].max() + 0.5,
    ]
)
ax.scatter(das_location["longitude"], das_location["latitude"], s=1, transform=ccrs.PlateCarree(), label="LAX DAS")
ax.legend()
plt.savefig(figure_path / "das_location.png", dpi=300)
plt.show()

## PhaseNet Picks

In [ ]:
pick_path = Path("picks_phasenet_das_raw")

In [ ]:
if not (result_path / "picks.csv").exists():
    picks = []

    for f in tqdm(sorted(list(pick_path.glob("*.csv")))):
        with open(f, "r") as fp:
            lines = fp.readlines()

        if len(lines) > 1:
            picks.append(pd.read_csv(io.StringIO("".join(lines))))

    picks = pd.concat(picks)
    picks["phase_time"] = pd.to_datetime(picks["phase_time"], utc=True)
    picks.to_csv(result_path / "picks.csv", index=False)

else:
    # picks = pd.read_csv(result_path / "picks.csv", parse_dates=["phase_time"])
    
    picks = pd.read_csv(result_path / "picks_0.6.csv", parse_dates=["phase_time"])
    # picks = picks[picks["phase_score"] > 0.6]
    # picks.to_csv(result_path / "picks_0.6.csv", index=False)

## Catalog

In [ ]:
if not (result_path / "events.csv").exists():
    client = Client("USGS")

    if not (result_path / "catalog.xml").exists():
        catalog = client.get_events(
            starttime=UTCDateTime(picks["phase_time"].min()),
            endtime=UTCDateTime(picks["phase_time"].max()),
            latitude=das_location["latitude"].mean(),
            longitude=das_location["longitude"].mean(),
            minradius=0,
            maxradius=2,
            limit=20000,
        )
        catalog.write(result_path / "catalog.xml", format="QUAKEML")

    else:
        catalog = obspy.read_events(result_path / "catalog.xml")

    print(f"{len(catalog) = }")

In [ ]:
def parase_catalog(catalog):
    events = {}
    for event in catalog:
        event_id = re.search(r"eventid=([a-z]+)(\d+)", event.resource_id.id).group(0).split("=")[1]
        events[event_id] = {
            "time": event.origins[0].time,
            "magnitude": event.magnitudes[0].mag,
            "latitude": event.origins[0].latitude,
            "longitude": event.origins[0].longitude,
            "depth_km": event.origins[0].depth / 1000,
        }

    return events


if not (result_path / "events.csv").exists():
    events = parase_catalog(catalog)
    events = pd.DataFrame.from_dict(events, orient="index")
    events["time"] = pd.to_datetime(events["time"], utc=True, format="%Y-%m-%dT%H:%M:%S.%fZ")
    events["distance_km"] = events.apply(
        lambda x: obspy.geodetics.base.gps2dist_azimuth(
            x.latitude, x.longitude, das_location["latitude"].mean(), das_location["longitude"].mean()
        )[0]
        / 1e3,
        axis=1,
    )
    events.to_csv(result_path / "events.csv", index_label="event_id")
else:
    events = pd.read_csv(result_path / "events.csv", parse_dates=["time"], index_col="event_id")

## DAS detactable events

In [ ]:
def calc_detectable_distance(magnitudes):
    scaling = {
        # "detectable_amplitude": -1.9,
        "detectable_amplitude": -1,
        # "detectable_amplitude": 0.0,
        "mean_site_term_S": 0.4,
        "mean_site_term_P": 0.4,
    }
    M_coef = (0.437, 0.69)
    D_coef = (-1.2693, -1.5875)

    detectable_amplitude = 10 ** scaling["detectable_amplitude"]
    mean_site_term_P = 10 ** scaling["mean_site_term_P"]
    mean_site_term_S = 10 ** scaling["mean_site_term_S"]

    

    D_sense_P = 10 ** (
        (-magnitudes * M_coef[0] + (np.log10(detectable_amplitude) - np.log10(mean_site_term_P))) / D_coef[0]
    )
    D_sense_S = 10 ** (
        (-magnitudes * M_coef[1] + (np.log10(detectable_amplitude) - np.log10(mean_site_term_S))) / D_coef[1]
    )

    return D_sense_S

detectable_distance_km = calc_detectable_distance(events["magnitude"].values)
events["detectable_distance_km"] = detectable_distance_km


In [ ]:
plt.figure()
# plt.plot(events["magnitude"], events["distance_km"], ".", markersize=1)

idx = (events["detectable_distance_km"] > events["distance_km"])
plt.plot(events[idx]["magnitude"], events[idx]["distance_km"], ".", color="red", markersize=1)
plt.plot(events[~idx]["magnitude"], events[~idx]["distance_km"], ".", color="gray", markersize=1)

# plt.plot(events["magnitude"], detectable_distance_km, ".", color="k", label="S")
xlim = plt.xlim()
plt.plot(np.arange(xlim[0], xlim[1], 0.1), calc_detectable_distance(np.arange(xlim[0], xlim[1], 0.1)), "--", color="C0")

plt.ylim([0, events["distance_km"].max()])
plt.xlabel("Magnitude")
plt.ylabel("Distance (km)")
# plt.legend()
plt.savefig(figure_path / "detectable_distance_km.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([minlongitude, maxlongitude, minlatitude, maxlatitude], crs=ccrs.PlateCarree())
terrain = cimgt.Stamen("terrain-background")
ax.add_image(terrain, 10, alpha=0.4)
ax.coastlines(resolution="10m", color="gray", linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

ax.scatter(
    events["longitude"],
    events["latitude"],
    s=3 ** events["magnitude"],
    alpha=0.5,
    color="gray",
    transform=ccrs.PlateCarree(),
    label="SCSN catalog"
)
idx = (events["detectable_distance_km"] > events["distance_km"])
ax.scatter(
    events[idx]["longitude"],
    events[idx]["latitude"],
    s=3 ** events[idx]["magnitude"],
    alpha=0.5,
    color="C3",
    transform=ccrs.PlateCarree(),
    label="Detectable events",
)
ax.scatter(das_location["longitude"], das_location["latitude"], s=3, transform=ccrs.PlateCarree(), label="LAX DAS")

ax.legend()
plt.savefig(figure_path / "catalog.png", dpi=300, bbox_inches="tight")
plt.show()

## PhaseNet picks

In [ ]:
plt.figure(figsize=(20, 5))

# x = pd.date_range(events["time"].min(), events["time"].max(), freq="1H")
# x = x.tz_convert("America/Los_Angeles")
# x_hour = x.hour
# y = np.array([das_location["channel"].max(), das_location["channel"].max()+500])
# x_hour, y = np.meshgrid(x_hour, y)
# z = - np.sin(2 * np.pi * x_hour/23)
# plt.pcolormesh(x, y, z, cmap="gray", shading="gouraud", alpha=0.05, rasterized=True)

idx = (events["detectable_distance_km"] > events["distance_km"])
plt.vlines(events[idx]["time"], das_location["channel"].max(), das_location["channel"].max()+500, color="k", alpha=0.5, linewidth=1.0, label="SCSN catalog", rasterized=True)


idx = (picks["phase_type"] == "P") & (picks["phase_score"] > 0.8)
plt.plot(picks["phase_time"][idx].iloc[::30], picks["channel_index"][idx].iloc[::30], ".", color="C3", alpha=1.0, markersize=1.0, linewidth=0.0, label="P picks", rasterized=True)

idx = (picks["phase_type"] == "S") & (picks["phase_score"] > 0.8)
plt.plot(picks["phase_time"][idx].iloc[::30], picks["channel_index"][idx].iloc[::30], ".", color="C0", alpha=1.0, markersize=1.0, linewidth=0.0, label="S picks", rasterized=True)

plt.xlabel("Time")
plt.ylabel("Channel index")

plt.legend(loc="lower right")
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(figure_path / "picks.png", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
gamma_catalog = pd.read_csv("catalog_gamma.csv")
gamma_catalog["time"] = pd.to_datetime(gamma_catalog["time"], utc=True)

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([minlongitude, maxlongitude, minlatitude, maxlatitude], crs=ccrs.PlateCarree())
terrain = cimgt.Stamen("terrain-background")
ax.add_image(terrain, 10, alpha=0.4)
ax.coastlines(resolution="10m", color="gray", linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

ax.scatter(
    gamma_catalog["longitude"],
    gamma_catalog["latitude"],
    s=5,
    alpha=0.5,
    color="C3",
    transform=ccrs.PlateCarree(),
    label="GaMMA catalog",
)
ax.scatter(das_location["longitude"], das_location["latitude"], s=3, transform=ccrs.PlateCarree(), label="LAX DAS")

ax.legend()
plt.savefig(figure_path / "gamma_catalog.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(
    [
        das_location["longitude"].min() - 0.2,
        das_location["longitude"].max() + 0.2,
        das_location["latitude"].min() - 0.2,
        das_location["latitude"].max() + 0.2,
    ]
)
terrain = cimgt.Stamen("terrain-background")
ax.add_image(terrain, 10, alpha=0.4)
ax.coastlines(resolution="10m", color="gray", linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

ax.scatter(
    gamma_catalog["longitude"],
    gamma_catalog["latitude"],
    s=1,
    alpha=0.5,
    color="C3",
    transform=ccrs.PlateCarree(),
    label="GaMMA catalog",
)
ax.scatter(das_location["longitude"], das_location["latitude"], s=3, transform=ccrs.PlateCarree(), label="LAX DAS")

ax.legend()
plt.savefig(figure_path / "gamma_catalog_zoomin.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
idx = (events["detectable_distance_km"] > events["distance_km"])
true_events = events[idx]
diff = true_events["time"].to_numpy()[None, :] - gamma_catalog["time"].to_numpy()[:, None]
recall_events = true_events[(np.abs(diff) < np.timedelta64(15, 's')).any(axis=0)]
# recall_events = true_events[(np.abs(diff) < np.timedelta64(30, 's')).any(axis=0)]

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([minlongitude, maxlongitude, minlatitude, maxlatitude], crs=ccrs.PlateCarree())
terrain = cimgt.Stamen("terrain-background")
ax.add_image(terrain, 10, alpha=0.4)
ax.coastlines(resolution="10m", color="gray", linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

idx = (events["detectable_distance_km"] > events["distance_km"])
ax.scatter(
    events[idx]["longitude"],
    events[idx]["latitude"],
    s=3 ** events[idx]["magnitude"],
    alpha=0.5,
    color="gray",
    transform=ccrs.PlateCarree(),
    label="Catalog events",
)

ax.scatter(
    recall_events["longitude"],
    recall_events["latitude"],
    s=3 ** recall_events["magnitude"],
    alpha=0.5,
    color="C3",
    transform=ccrs.PlateCarree(),
    label="Detected events",
)

ax.scatter(das_location["longitude"], das_location["latitude"], s=3, transform=ccrs.PlateCarree(), label="LAX DAS")

ax.legend()
plt.savefig(figure_path / "catalog_recall.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(
    [
        das_location["longitude"].min() - 0.2,
        das_location["longitude"].max() + 0.2,
        das_location["latitude"].min() - 0.2,
        das_location["latitude"].max() + 0.2,
    ]
)
terrain = cimgt.Stamen("terrain-background")
ax.add_image(terrain, 10, alpha=0.4)
ax.coastlines(resolution="10m", color="gray", linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

idx = (events["detectable_distance_km"] > events["distance_km"])
ax.scatter(
    events[idx]["longitude"],
    events[idx]["latitude"],
    s=5 ** events[idx]["magnitude"],
    alpha=0.5,
    color="gray",
    transform=ccrs.PlateCarree(),
    label="Catalog events",
)

ax.scatter(
    recall_events["longitude"],
    recall_events["latitude"],
    s=5 ** recall_events["magnitude"],
    alpha=0.5,
    color="C3",
    transform=ccrs.PlateCarree(),
    label="Detected events",
)

ax.scatter(das_location["longitude"], das_location["latitude"], s=3, transform=ccrs.PlateCarree(), label="LAX DAS")

# scale legend
ax.legend(markerscale=0.5)
plt.savefig(figure_path / "catalog_recall_zoomin.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
gamma_picks = []
for pick in tqdm(sorted(list(Path("gamma").glob("*.csv")))):
    tmp = pd.read_csv(pick)
    tmp["file_id"] = pick.stem
    gamma_picks.append(tmp)
gamma_picks = pd.concat(gamma_picks)

In [ ]:
gamma_picks.to_csv(result_path / "gamma_picks.csv", index=False)

In [ ]:
gamma_picks["phase_time"] = pd.to_datetime(gamma_picks["phase_time"], utc=True)

In [ ]:
plt.figure(figsize=(20, 5))

# x = pd.date_range(events["time"].min(), events["time"].max(), freq="1H")
# x = x.tz_convert("America/Los_Angeles")
# x_hour = x.hour
# y = np.array([das_location["channel"].max(), das_location["channel"].max()+500])
# x_hour, y = np.meshgrid(x_hour, y)
# z = - np.sin(2 * np.pi * x_hour/23)
# plt.pcolormesh(x, y, z, cmap="gray", shading="gouraud", alpha=0.05, rasterized=True)

idx = (events["detectable_distance_km"] > events["distance_km"])
plt.vlines(events[idx]["time"], das_location["channel"].max(), das_location["channel"].max()+500, color="k", alpha=0.5, linewidth=1.0, label="SCSN catalog", rasterized=True)


idx = (gamma_picks["phase_type"] == "P")
plt.plot(gamma_picks["phase_time"][idx].iloc[::30], gamma_picks["channel_index"][idx].iloc[::30], ".", color="C3", alpha=1.0, markersize=1.0, linewidth=0.0, label="P picks", rasterized=True)

idx = (gamma_picks["phase_type"] == "S")
plt.plot(gamma_picks["phase_time"][idx].iloc[::30], gamma_picks["channel_index"][idx].iloc[::30], ".", color="C0", alpha=1.0, markersize=1.0, linewidth=0.0, label="S picks", rasterized=True)

plt.xlabel("Time")
plt.ylabel("Channel index")

plt.legend(loc="lower right")
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(figure_path / "gamma_picks.png", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
def parse_inventory(inventory, mseed_ids=None):
    stations = {}
    num = 0
    for net in inventory:
        for sta in net:
            components = defaultdict(list)
            channel = {}

            for chn in sta:
                key = f"{chn.location_code}{chn.code[:-1]}"
                components[key].append(chn.code[-1])

                if key not in channel:
                    channel[key] = {
                        "latitude": chn.latitude,
                        "longitude": chn.longitude,
                        "elevation_m": chn.elevation,
                        "location": chn.location_code,
                        "device": chn.code[:-1],
                    }

            for key in components:
                station_id = f"{net.code}.{sta.code}.{channel[key]['location']}.{channel[key]['device']}"
                if (mseed_ids is not None) and (station_id not in mseed_ids):
                    continue
                num += 1
                stations[station_id] = {
                    "network": net.code,
                    "station": sta.code,
                    "location": channel[key]["location"],
                    "component": sorted(components[key]),
                    "latitude": channel[key]["latitude"],
                    "longitude": channel[key]["longitude"],
                    "elevation_m": channel[key]["elevation_m"],
                    "depth_km": - channel[key]["elevation_m"] / 1e3,
                }
                
    print(f"Parse {num} stations")

    return stations

In [ ]:
client = Client("SCEDC")
inventory = client.get_stations(
    starttime=UTCDateTime(picks["phase_time"].min()),
    endtime=UTCDateTime(picks["phase_time"].max()),
    latitude=das_location["latitude"].mean(),
    longitude=das_location["longitude"].mean(),
    minradius=0,
    maxradius=2,
    level="channel",
)
inventory.write(result_path / "inventory.xml", format="STATIONXML") 

In [ ]:
stations = parse_inventory(inventory)

In [ ]:
stations = parse_inventory(inventory)
with open(result_path / "stations.json", "w") as f:
    json.dump(stations, f, indent=4)
stations = pd.DataFrame.from_dict(stations, orient="index")
stations.to_csv(result_path / "stations.csv", index_label="station_id")

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution="10m", color="gray", linewidth=0.5)
terrain = cimgt.Stamen("terrain-background")
ax.add_image(terrain, 10)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False
ax.set_extent(
    [
        das_location["longitude"].min() - 0.5,
        das_location["longitude"].max() + 0.5,
        das_location["latitude"].min() - 0.5,
        das_location["latitude"].max() + 0.5,
    ]
)
ax.scatter(stations["longitude"], stations["latitude"], s=15, marker="^", transform=ccrs.PlateCarree(), label="Seismometer")
ax.legend()
plt.savefig(figure_path / "seismic_network.png", dpi=300)
plt.show()

In [ ]:
raise

## Save picks by day

In [ ]:
output_dir = Path("picks_by_day")
if not output_dir.exists():
    output_dir.mkdir()

In [ ]:
# for i in pd.date_range(events["time"].min(), events["time"].max(), freq="1D")

# group picks by day
picks["day"] = picks["phase_time"].dt.date

# group picks by day and save to separate csv files
for day, group in tqdm(picks.groupby("day")):
    group.to_csv(output_dir / f"picks_{day}.csv", index=False)


In [ ]:
picks["day"]

In [ ]:
plt.figure(figsize=(50, 10))

x = pd.date_range(events["time"].min(), events["time"].max(), freq="1H")
x = x.tz_convert("America/Los_Angeles")
x_hour = x.hour
y = np.array([das_location["channel"].max(), das_location["channel"].max()+500])
x_hour, y = np.meshgrid(x_hour, y)
z = - np.sin(2 * np.pi * x_hour/23)
plt.pcolormesh(x, y, z, cmap="gray", shading="gouraud", alpha=0.05, rasterized=True)

idx = (events["detectable_distance_km"] > events["distance_km"])
plt.vlines(events[idx]["time"], das_location["channel"].max(), das_location["channel"].max()+500, color="C2", alpha=1.0, linewidth=1)

## color the background based based on events["time"] to indicate day/night
t = pd.date_range(events["time"].min(), events["time"].max(), freq="1H")


idx = (picks["phase_type"] == "P") & (picks["phase_score"] > 0.9)
plt.plot(picks["phase_time"][idx].iloc[::1], picks["channel_index"][idx].iloc[::1], ".", color="C3", alpha=1.0, markersize=1)

idx = (picks["phase_type"] == "S") & (picks["phase_score"] > 0.9)
plt.plot(picks["phase_time"][idx].iloc[::1], picks["channel_index"][idx].iloc[::1], ".", color="C0", alpha=1.0, markersize=1)

plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(figure_path / "picks.png", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
dt = picks["phase_time"].dt.tz_convert("America/Los_Angeles").dt
plt.figure()
plt.hist(dt.hour, bins=np.arange(0, 25, 1), edgecolor="white", color="C0")
plt.xticks(np.arange(0, 25, 3))
plt.xlim([0, 24])
plt.grid(linestyle="--", alpha=0.5)
plt.xlabel("Hour")
plt.ylabel("Number of picks")
plt.savefig(figure_path / "picks_hour.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
picks["phase_time"].dt.tz_convert("America/Los_Angeles").dt.hour

In [ ]:
picks["phase_time"]

In [ ]:
plt.figure(figsize=(10, 3))
plt.pcolormesh(x, y, z, cmap="gray", shading="gouraud")
plt.xlim([events["time"].min(), events["time"].min() + pd.Timedelta(days=1)])
plt.show()

In [ ]:
das_location

In [ ]:
plt.figure(figsize=(50, 10))
idx = (events["detectable_distance_km"] > events["distance_km"])
plt.vlines(events[idx]["time"], das_location["channel"].min(), das_location["channel"].max(), color="gray", alpha=0.5)
# plt.plot(picks["phase_time"][idx].iloc[::1], picks["channel_index"][idx].iloc[::1], ".", color="r", markersize=1)
# idx = (picks["phase_type"] == "S") & (picks["phase_score"] > 0.8)
# plt.plot(picks["phase_time"][idx].iloc[::1], picks["channel_index"][idx].iloc[::1], ".", color="b", markersize=1)
# plt.savefig(figure_path / "picks.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
picks["phase_time"].min()

In [ ]:
# add tz_localize("UTC") to phase_time
picks["phase_time"] = pd.to_datetime(picks["phase_time"], utc=True)  # .dt.tz_localize("UTC")

In [ ]:
picks

In [ ]:
# picks

In [ ]:
# plt.figure()
# plt.hist(picks["phase_index"])
# plt.show()

In [ ]:
plt.figure()
plt.hist(picks["channel_index"])
plt.show()

In [ ]:
plt.figure()
plt.hist(picks["phase_time"])
plt.show()

In [ ]:
client = Client("SCEDC")
event = client.get_events(
    starttime=picks["phase_time"].min(),
    endtime=picks["phase_time"].max(),
    latitude=
    longitude=
    minmagnitude=5,
    
)
print_event(event)

In [ ]:
plt.figure(figsize=(50, 10))
idx = (picks["phase_type"] == "P") & (picks["phase_score"] > 0.8)
plt.plot(picks["phase_time"][idx].iloc[::1], picks["channel_index"][idx].iloc[::1], ".", color="r", markersize=1)
idx = (picks["phase_type"] == "S") & (picks["phase_score"] > 0.8)
plt.plot(picks["phase_time"][idx].iloc[::1], picks["channel_index"][idx].iloc[::1], ".", color="b", markersize=1)
plt.savefig("picks.png", dpi=600, bbox_inches="tight")
plt.show()

In [ ]:
picks["phase_time"].hist()